In [185]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input,Dense,Flatten
from keras.layers import LSTM
from numpy import array

In [186]:
df_train = pd.read_csv('./Train_X.csv',encoding= 'cp950')

In [264]:
df_train = df_train.sort_values('日期')

stock_list_train = (df_train['股票代號'].unique()).tolist()

data_train = {}
data_test ={}
for name in stock_list_train:
    data_train[name]=df_train[df_train['股票代號']==name].loc[:,['開盤價', '最高價','最低價','收盤價','成交量.股.']]

In [376]:
#定義正規化函式
def normalize(train):
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

#定義餵入的feature跟label序列
def train_(df, ref_day, predict_day):
    X_train, Y_train = [], []
    for i in range(df.shape[0]+predict_day-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:]))
        Y_train.append(np.array(df.iloc[i+ref_day : i+ref_day+predict_day,0:6]))
    return np.array(X_train), np.array(Y_train)

TWA00_train= normalize(data_train['TWA00'])
TWA00_train.head(10)

,開盤價,最高價,最低價,收盤價,成交量.股.
30182,-0.290081,-0.302318,-0.276437,-0.285034,-0.134711
28504,-0.316448,-0.339877,-0.314145,-0.335621,-0.148049
26825,-0.341165,-0.352770,-0.351505,-0.371284,-0.112559
25145,-0.370475,-0.369217,-0.333546,-0.361864,-0.128461
23465,-0.344583,-0.338049,-0.305986,-0.328362,0.097388
21784,-0.338984,-0.352090,-0.338313,-0.360040,0.292768
20103,-0.336352,-0.359849,-0.326563,-0.348796,-0.062034
18422,-0.314307,-0.322305,-0.298130,-0.320552,-0.094501
16740,-0.347036,-0.352715,-0.325075,-0.334394,-0.185551
15056,-0.329632,-0.312671,-0.291413,-0.292691,0.067167


In [377]:
X_train,Y_train=train_(TWA00_train,7,1)

In [381]:
matrix = np.empty([231,1,5], dtype=float)
for k in range(230):
    matrix[k] = Y_train[k]
    print(Y_train[k])

[[-0.31430678 -0.32230504 -0.29812996 -0.32055189 -0.09450059]]
[[-0.34703609 -0.35271486 -0.32507534 -0.33439426 -0.18555054]]
[[-0.32963185 -0.3126711  -0.29141259 -0.29269086  0.06716684]]
[[-0.26287278 -0.27408064 -0.25920716 -0.28536345  0.01008807]]
[[-0.28237954 -0.30540463 -0.28178815 -0.29771121 -0.12108865]]
[[-0.3079992  -0.29966176 -0.27874446 -0.27812035 -0.15743862]]
[[-0.27364238 -0.28877768 -0.26705548 -0.294722   -0.0517191 ]]
[[-0.30535936 -0.30172451 -0.26926424 -0.27525377 -0.18797331]]
[[-0.26986562 -0.29055132 -0.2706988  -0.29840103 -0.23745239]]
[[-0.31897906 -0.32120335 -0.28433088 -0.29594068 -0.34543124]]
[[-0.36726705 -0.39086777 -0.38676128 -0.39998072 -0.24288088]]
[[-0.54583376 -0.55155066 -0.5474696  -0.55180989 -0.04536789]]
[[-0.54895639 -0.53613479 -0.50624697 -0.51614624 -0.15243362]]
[[-0.47277489 -0.47716694 -0.45147574 -0.45763426 -0.16165533]]
[[-0.44752123 -0.44748378 -0.42489469 -0.45410852 -0.25062898]]
[[-0.44951473 -0.45349229 -0.41761564 -0

In [382]:
matrix[2]

array([[-0.32963185, -0.3126711 , -0.29141259, -0.29269086,  0.06716684]])

In [364]:
a = Y_train.reshape(231,5)

ValueError: cannot reshape array of size 231 into shape (231,5)

In [311]:
# define model
import keras
units = 50
inputs1 = Input(shape=(X_train.shape[1], X_train.shape[2]))
lstm1=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(inputs1)
lstm2=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(lstm1)
lstm3=LSTM(units, activation='relu', recurrent_activation='hard_sigmoid',return_sequences=True)(lstm2)
final=LSTM(5, activation='tanh', recurrent_activation='hard_sigmoid',return_sequences = False)(lstm3)
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.99, beta_2=0.999, amsgrad=False)

model = Model(inputs=inputs1, outputs=final)
model.compile(optimizer=adam, loss='mean_squared_error')


In [312]:
model.fit(X_train, a, epochs = 100, batch_size = 20)

Epoch 1/100
231/231 [==============================] - 3s 14ms/step - loss: 0.0210
Epoch 2/100
231/231 [==============================] - 0s 1ms/step - loss: 0.0024
Epoch 3/100
231/231 [==============================] - 0s 1ms/step - loss: 8.3869e-04
Epoch 4/100
231/231 [==============================] - 0s 1ms/step - loss: 3.8689e-04
Epoch 5/100
231/231 [==============================] - 0s 1ms/step - loss: 4.4626e-04
Epoch 6/100
231/231 [==============================] - 0s 1ms/step - loss: 2.6713e-04
Epoch 7/100
231/231 [==============================] - 0s 1ms/step - loss: 1.7054e-04
Epoch 8/100
231/231 [==============================] - 0s 1ms/step - loss: 2.4638e-04
Epoch 9/100
231/231 [==============================] - 0s 1ms/step - loss: 2.0679e-04
Epoch 10/100
231/231 [==============================] - 0s 1ms/step - loss: 8.2310e-05
Epoch 11/100
231/231 [==============================] - 0s 1ms/step - loss: 8.0304e-05
Epoch 12/100
231/231 [==============================] - 0s 

231/231 [==============================] - 0s 1ms/step - loss: 3.7738e-08
Epoch 96/100
231/231 [==============================] - 0s 1ms/step - loss: 3.5220e-08
Epoch 97/100
231/231 [==============================] - 0s 1ms/step - loss: 3.5488e-08
Epoch 98/100
231/231 [==============================] - 0s 1ms/step - loss: 3.4919e-08
Epoch 99/100
231/231 [==============================] - 0s 1ms/step - loss: 3.3833e-08
Epoch 100/100
231/231 [==============================] - 0s 1ms/step - loss: 3.3994e-08


In [277]:
def train_1(df, ref_day, predict_day):
    X_train, Y_train = [], []
    for i in range(df.shape[0]+predict_day-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day,0:6]))
    return np.array(X_train), np.array(Y_train)

In [278]:
df_test = pd.read_csv('./Test_X.csv',encoding= 'cp950')


for name in stock_list_train:
    data_test[name]=df_test[df_test['股票代號']==name].loc[:,['開盤價', '最高價','最低價','收盤價','成交量.股.']]
TWA00_test= normalize(data_test['TWA00'])
X_test,Y_test=train_1(TWA00_test,7,1)

In [279]:
TWA00_test

,開盤價,最高價,最低價,收盤價,成交量.股.
1656,0.452203,0.429496,0.413304,0.314309,-0.178195
3438,0.238209,0.366352,0.293378,0.358096,-0.014445
5219,0.253844,0.150133,0.243354,0.191826,-0.529366
6999,0.122968,0.112905,0.142945,0.180959,-0.281233
8777,0.161589,0.071974,0.130970,0.045132,-0.163874
10557,-0.072285,0.041691,-0.017519,0.112543,0.304949
12337,-0.001618,-0.080947,-0.025424,-0.104482,0.033537
14115,-0.238106,-0.178741,-0.205352,-0.111130,0.202569
15894,-0.369008,-0.342360,-0.388961,-0.345350,0.470634
17671,-0.547797,-0.570504,-0.586696,-0.641904,0.155424


In [280]:
X_test.shape

(4, 7, 5)

In [284]:
matrix = np.zeros((4,1,5))
matrix.shape

(4, 1, 5)

In [290]:
matrix[0][0] = Y_test[0]
matrix

array([[[-0.23810572, -0.17874068, -0.20535224, -0.11112965,
          0.20256904]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]]])

In [285]:
Y_test[0]

array([[-0.23810572, -0.17874068, -0.20535224, -0.11112965,  0.20256904]])

In [254]:
X_test[2]

array([[ 0.25384396,  0.15013277,  0.24335441,  0.19182632, -0.52936623],
       [ 0.12296801,  0.11290471,  0.1429449 ,  0.18095916, -0.28123302],
       [ 0.16158884,  0.07197426,  0.1309704 ,  0.04513185, -0.16387352],
       [-0.07228463,  0.04169135, -0.01751853,  0.11254262,  0.30494918],
       [-0.00161783, -0.08094679, -0.02542377, -0.10448179,  0.03353686],
       [-0.23810572, -0.17874068, -0.20535224, -0.11112965,  0.20256904],
       [-0.36900756, -0.34236033, -0.38896117, -0.34535018,  0.47063377]])

In [256]:
predict_y = model.predict(X_test)

In [257]:
predict_y

array([[-0.08440772, -0.06004865, -0.10839376, -0.0880492 ,  0.16609287],
       [-0.14595668, -0.11808517, -0.15015912, -0.13611495,  0.05663164],
       [-0.31256953, -0.27866918, -0.3269158 , -0.29539642,  0.18631744],
       [-0.49309927, -0.4633901 , -0.47412276, -0.45190802,  0.0425681 ]],
      dtype=float32)

In [258]:
def denormalize(train):
    denorm = train*(np.max(data_train['TWA00']['收盤價'])-np.min(data_train['TWA00']['收盤價']))+np.mean(data_train['TWA00']['收盤價'])
    return denorm

In [262]:
Y_test = pd.DataFrame(Y_test)
test_y = denormalize(Y_test)
z = denormalize(predict_y)

In [263]:
test_y

,0
0,"[[8673.766697966548, 8751.219665810153, 8716.4..."
1,"[[8502.98037238154, 8537.746743738862, 8476.94..."
2,"[[8269.715363321695, 8240.090601618547, 8218.9..."
3,[]


In [261]:
import matplotlib.pyplot as plt 
plt.plot(test_y, color = 'red', label = 'Real Price')  # 紅線表示真實股價
plt.plot(z, color = 'blue', label = 'Predicted Price')  # 藍線表示預測股價
plt.title('Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

ValueError: setting an array element with a sequence.